Instructions:
-

1. Read the article: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
2. Replicate the study using the same dataset.
3. Read articles about Adjusted Rand Index, Normalized Mutual Information, and Folkes-Mallows Index (only use paper published in IEEE, sciencedirect, springerlink, Taylor Francis).
4. Aside from the Adjusted Rand Index (ARI), and Normalized Mutual Information (NMI), use the Folkes-Mallows Index (FMI), and compare the result of each performance index.
5. Compare and contrast each performance index, what are the advantages and disadvantages of ARI, NMI, and FMI, and when to use each?
6. Using Kmodes and Hierarchical Clustering, use the same dataset and perform categorical data clustering, use FMI, ARI, and NMI for the comparison of performance.
7. Write your report using Latex. Your report should be focused on the "why's and the what's" of each performance metrices (i.e. why is FMI always greater than ARI and NMI? What's the problem with ARI and NMI?).

In [16]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

soybean= fetch_ucirepo(id=91) 
zoo = fetch_ucirepo(id=111) 
heart_disease = fetch_ucirepo(id=45) 
dermatology = fetch_ucirepo(id=33)
breast_cancer = fetch_ucirepo(id=15)
mushroom = fetch_ucirepo(id=73) 

X = soybean.data.features
y = soybean.data.targets 
soybean_data = pd.merge(X, y, left_index=True, right_index=True)

X = zoo.data.features
y = zoo.data.targets 
zoo_data = pd.merge(X, y, left_index=True, right_index=True)

X = heart_disease.data.features
y = heart_disease.data.targets 
heart_disease_data = pd.merge(X, y, left_index=True, right_index=True)

X = dermatology.data.features
y = dermatology.data.targets 
dermatology_data = pd.merge(X, y, left_index=True, right_index=True)

X = breast_cancer.data.features
y = breast_cancer.data.targets 
breast_cancer_data = pd.merge(X, y, left_index=True, right_index=True)

X = mushroom.data.features
y = mushroom.data.targets 
mushroom_data = pd.merge(X, y, left_index=True, right_index=True)

In [44]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score
from sklearn.cluster import KMeans
from sklearn.manifold import SpectralEmbedding
from sklearn.preprocessing import OneHotEncoder
import networkx as nx
import itertools
import warnings
from sklearn.cluster import AgglomerativeClustering
from kmodes.kmodes import KModes
from sklearn.preprocessing import LabelEncoder 

In [18]:
soybean_data.head(10)

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,4,0,2,1,1,1,0,1,0,2,...,0,0,4,0,0,0,0,0,0,D1
1,5,0,2,1,0,3,1,1,1,2,...,0,0,4,0,0,0,0,0,0,D1
2,3,0,2,1,0,2,0,2,1,1,...,0,0,4,0,0,0,0,0,0,D1
3,6,0,2,1,0,1,1,1,0,0,...,0,0,4,0,0,0,0,0,0,D1
4,4,0,2,1,0,3,0,2,0,2,...,0,0,4,0,0,0,0,0,0,D1
5,5,0,2,1,0,2,0,1,1,0,...,0,0,4,0,0,0,0,0,0,D1
6,3,0,2,1,0,2,1,1,0,1,...,0,0,4,0,0,0,0,0,0,D1
7,3,0,2,1,0,1,0,2,1,2,...,0,0,4,0,0,0,0,0,0,D1
8,6,0,2,1,0,3,0,1,1,1,...,0,0,4,0,0,0,0,0,0,D1
9,6,0,2,1,0,1,0,1,0,2,...,0,0,4,0,0,0,0,0,0,D1


In [19]:
zoo_data.head(10)

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
5,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
6,1,0,0,1,0,0,0,1,1,1,0,0,4,1,1,1,1
7,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,4
8,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
9,1,0,0,1,0,0,0,1,1,1,0,0,4,0,1,0,1


In [20]:
heart_disease_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
5,56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0
6,62,0,4,140,268,0,2,160,0,3.6,3,2.0,3.0,3
7,57,0,4,120,354,0,0,163,1,0.6,1,0.0,3.0,0
8,63,1,4,130,254,0,2,147,0,1.4,2,1.0,7.0,2
9,53,1,4,140,203,1,2,155,1,3.1,3,0.0,7.0,1


In [22]:
dermatology_data.head(10)

,erythema,scaling,definite-borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral-mucosal involvement,knee elbow involvement,scalp involvement,...,disappearance of the granular layer,vacuolisation and damage of the basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear infiltrate,band-like infiltrate,age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55.0,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8.0,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26.0,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40.0,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45.0,3
5,2,3,2,0,0,0,0,0,0,0,...,0,0,2,0,0,0,1,0,41.0,2
6,2,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,18.0,5
7,2,2,3,3,3,3,0,2,0,0,...,2,2,3,2,0,0,3,3,57.0,3
8,2,2,1,0,2,0,0,0,0,0,...,0,0,2,0,0,0,2,0,22.0,4
9,2,2,1,0,1,0,0,0,0,0,...,0,0,2,0,0,0,2,0,30.0,4


In [23]:
breast_cancer_data.head(10)

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
5,8,10,10,8,7,10.0,9,7,1,4
6,1,1,1,1,2,10.0,3,1,1,2
7,2,1,2,1,2,1.0,3,1,1,2
8,2,1,1,1,2,1.0,1,1,5,2
9,4,2,1,1,2,1.0,2,1,1,2


In [24]:
mushroom_data.head(10)

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,poisonous
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
5,x,y,y,t,a,f,c,b,n,e,...,w,w,p,w,o,p,k,n,g,e
6,b,s,w,t,a,f,c,b,g,e,...,w,w,p,w,o,p,k,n,m,e
7,b,y,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,s,m,e
8,x,y,w,t,p,f,c,n,p,e,...,w,w,p,w,o,p,k,v,g,p
9,b,s,y,t,a,f,c,b,g,e,...,w,w,p,w,o,p,k,s,m,e


In [25]:
soybean_data = soybean_data.dropna()
zoo_data = zoo_data.dropna()
heart_disease_data = heart_disease_data.dropna()
dermatology_data = dermatology_data.dropna()
breast_cancer_data = breast_cancer_data.dropna()
mushroom_data = mushroom_data.dropna()

In [42]:

def jaccard_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

def ochiai_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    denominator = np.sqrt(len(set1) * len(set2))
    return intersection / denominator

def overlap_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    min_length = min(len(set1), len(set2))
    return intersection / min_length

def dice_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    denominator = len(set1) + len(set2)
    return 2 * intersection / denominator

def graph_based_representation(data):
    num_samples, num_features = data.shape
    similarity_matrix = np.zeros((num_features, num_features))
    for i, j in itertools.combinations(range(num_features), 2):
        similarity_matrix[i, j] = jaccard_coefficient(set(data[:, i]), set(data[:, j]))
        similarity_matrix[j, i] = similarity_matrix[i, j]
    G = nx.from_numpy_array(similarity_matrix)
    embedding = SpectralEmbedding(n_components=p)
    representation_matrix = embedding.fit_transform(similarity_matrix)
    return representation_matrix

def joint_operation(data, representation_matrix):
    return np.dot(data, representation_matrix)

def mean_operation(data, representation_matrix):
    return np.mean(np.dot(data, representation_matrix), axis=1)

def perform_clustering(data, k):
    kmeans = KMeans(n_clusters=k)
    return kmeans.fit_predict(data)

In [47]:
p = 10
q = 10 
k = 3

results = []

datasets = ["soybean_data", "zoo_data", "heart_disease_data", "dermatology_data", "breast_cancer_data", "mushroom_data"]
for dataset_name in datasets:
    dataset = globals()[dataset_name]
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        try:
            X = dataset
            enc = OneHotEncoder()
            X_encoded = enc.fit_transform(X)
            representation_matrix = graph_based_representation(X_encoded.toarray())
            integrated_data = joint_operation(X_encoded.toarray(), representation_matrix)
            
            # Explicitly set n_init to suppress the warning
            kmeans = KMeans(n_clusters=k, n_init=10)
            labels = kmeans.fit_predict(integrated_data)

            true_labels = dataset.iloc[:, -1] 
            ARI = adjusted_rand_score(true_labels, labels)
            NMI = normalized_mutual_info_score(true_labels, labels)
            FMI = fowlkes_mallows_score(true_labels, labels)
            results.append([dataset_name, ARI, NMI, FMI])
        except UserWarning as e:
            print(f"Warning: {e}")
            

results_df = pd.DataFrame(results, columns=["Dataset", "ARI", "NMI", "FMI"])
print(results_df)


              Dataset       ARI       NMI       FMI
0        soybean_data  0.780695  0.877931  0.854668
1            zoo_data  0.573437  0.588278  0.700263
2  heart_disease_data  0.063009  0.086893  0.402222
3    dermatology_data  0.208840  0.359863  0.446715
4  breast_cancer_data  0.580242  0.494337  0.788131
5       mushroom_data  0.311113  0.371810  0.647707


5. Compare and contrast each performance index, what are the advantages and disadvantages of ARI, NMI, and FMI, and when to use each?


The Adjusted Rand Index (ARI) quantifies the similarity between two clusterings by considering all pairs of samples and counting pairs that are assigned to the same or different clusters in both the predicted and true clusterings. While it provides a range from -1 to 1, where 1 indicates perfect similarity, ARI is sensitive to the number of clusters and the dataset's size. It's particularly useful when the number of clusters is unknown but can produce high values even for random clusterings.

Normalized Mutual Information (NMI) measures the mutual information between the true and predicted clusterings, normalized by the entropy of both. With values between 0 and 1, NMI offers insight into agreement levels, especially when clusters vary in size. However, it might favor algorithms generating numerous clusters and isn't sensitive to the actual number of clusters formed.

The Fowlkes-Mallows Index (FMI) assesses clustering similarity by examining pairs of samples correctly or incorrectly grouped together in the predicted and true clusterings. Despite its intuitive interpretation and a scale from 0 to 1, where 1 indicates perfect agreement, FMI shares ARI's sensitivity to cluster count and dataset size.

Choosing the appropriate metric depends on your clustering goals and dataset characteristics. ARI is best for scenarios where ground truth labels are available, and the number of clusters is uncertain. NMI provides normalized agreement measures, while FMI offers simplicity with a focus on sample pairs. In your results, differences in performance across datasets highlight the importance of selecting the most suitable metric based on your specific clustering requirements.

Using Kmodes and Hierarchical Clustering, use the same dataset and perform categorical data clustering, use FMI, ARI, and NMI for the comparison of performance.

In [48]:
results_categorical = []

for dataset_name in datasets:
    
    dataset = globals()[dataset_name]
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        try:
            X_cat = dataset.iloc[:, :-1]
            true_labels_cat = dataset.iloc[:, -1]

            encoder = LabelEncoder()
            X_cat_encoded = X_cat.apply(encoder.fit_transform)

            km = KModes(n_clusters=k, init='Huang', n_init=5, verbose=0)
            km_labels = km.fit_predict(X_cat_encoded)
            ARI_km = adjusted_rand_score(true_labels_cat, km_labels)
            NMI_km = normalized_mutual_info_score(true_labels_cat, km_labels)
            FMI_km = fowlkes_mallows_score(true_labels_cat, km_labels)

            ac = AgglomerativeClustering(n_clusters=k, linkage='ward')
            ac_labels = ac.fit_predict(X_cat_encoded)
            ARI_ac = adjusted_rand_score(true_labels_cat, ac_labels)
            NMI_ac = normalized_mutual_info_score(true_labels_cat, ac_labels)
            FMI_ac = fowlkes_mallows_score(true_labels_cat, ac_labels)
            
            results_categorical.append([dataset_name + " (Kmodes)", ARI_km, NMI_km, FMI_km])
            results_categorical.append([dataset_name + " (Hierarchical)", ARI_ac, NMI_ac, FMI_ac])
        except UserWarning as e:
            print(f"Warning: {e}")

results_categorical_df = pd.DataFrame(results_categorical, columns=["Dataset", "ARI", "NMI", "FMI"])

print(results_categorical_df)

                              Dataset       ARI       NMI       FMI
0               soybean_data (Kmodes)  0.653689  0.837666  0.783908
1         soybean_data (Hierarchical)  0.653689  0.837666  0.783908
2                   zoo_data (Kmodes)  0.721570  0.714510  0.812035
3             zoo_data (Hierarchical)  0.461701  0.585056  0.645736
4         heart_disease_data (Kmodes)  0.099396  0.176940  0.415937
5   heart_disease_data (Hierarchical)  0.009543  0.010944  0.353434
6           dermatology_data (Kmodes)  0.380771  0.431111  0.560634
7     dermatology_data (Hierarchical)  0.032201  0.078147  0.293614
8         breast_cancer_data (Kmodes)  0.482607  0.510921  0.722824
9   breast_cancer_data (Hierarchical)  0.781678  0.688725  0.893555
10             mushroom_data (Kmodes)  0.301662  0.316257  0.607032
11       mushroom_data (Hierarchical)  0.298722  0.431275  0.610515


In [49]:
categorical_results = []

for dataset_name in datasets:
    
    dataset = globals()[dataset_name]
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        try:
            X_categorical = dataset.iloc[:, :-1]
            true_labels_categorical = dataset.iloc[:, -1]

            label_encoder = LabelEncoder()
            X_categorical_encoded = X_categorical.apply(label_encoder.fit_transform)

            kmodes_clustering = KModes(n_clusters=k, init='Huang', n_init=5, verbose=0)
            kmodes_labels = kmodes_clustering.fit_predict(X_categorical_encoded)
            ARI_kmodes = adjusted_rand_score(true_labels_categorical, kmodes_labels)
            NMI_kmodes = normalized_mutual_info_score(true_labels_categorical, kmodes_labels)
            FMI_kmodes = fowlkes_mallows_score(true_labels_categorical, kmodes_labels)

            hierarchical_clustering = AgglomerativeClustering(n_clusters=k, linkage='ward')
            hierarchical_labels = hierarchical_clustering.fit_predict(X_categorical_encoded)
            ARI_hierarchical = adjusted_rand_score(true_labels_categorical, hierarchical_labels)
            NMI_hierarchical = normalized_mutual_info_score(true_labels_categorical, hierarchical_labels)
            FMI_hierarchical = fowlkes_mallows_score(true_labels_categorical, hierarchical_labels)
            
            categorical_results.append([dataset_name + " (KModes)", ARI_kmodes, NMI_kmodes, FMI_kmodes])
            categorical_results.append([dataset_name + " (Hierarchical)", ARI_hierarchical, NMI_hierarchical, FMI_hierarchical])
        except UserWarning as e:
            print(f"Warning: {e}")

categorical_results_df = pd.DataFrame(categorical_results, columns=["Dataset", "ARI", "NMI", "FMI"])

print(categorical_results_df)


                              Dataset       ARI       NMI       FMI
0               soybean_data (KModes)  0.653689  0.837666  0.783908
1         soybean_data (Hierarchical)  0.653689  0.837666  0.783908
2                   zoo_data (KModes)  0.721570  0.714510  0.812035
3             zoo_data (Hierarchical)  0.461701  0.585056  0.645736
4         heart_disease_data (KModes)  0.265450  0.212498  0.527421
5   heart_disease_data (Hierarchical)  0.009543  0.010944  0.353434
6           dermatology_data (KModes)  0.298371  0.384206  0.504422
7     dermatology_data (Hierarchical)  0.032201  0.078147  0.293614
8         breast_cancer_data (KModes)  0.482945  0.517182  0.721153
9   breast_cancer_data (Hierarchical)  0.781678  0.688725  0.893555
10             mushroom_data (KModes)  0.315965  0.335031  0.635397
11       mushroom_data (Hierarchical)  0.298722  0.431275  0.610515


7. Write your report using Latex. Your report should be focused on the "why's and the what's" of each performance metrices (i.e. why is FMI always greater than ARI and NMI? What's the problem with ARI and NMI?).

Adjusted Rand Index (ARI) quantifies clustering similarity by considering all pairs of samples and comparing how they are clustered in both the predicted and true clusterings. However, ARI can produce high values even for random clusterings and is sensitive to the number of clusters and dataset size.

Normalized Mutual Information (NMI) measures the mutual information between true and predicted clusterings, normalized by their respective entropies. NMI provides insights into agreement levels, especially when clusters vary in size. However, it may favor algorithms generating numerous clusters and isn't sensitive to the actual number of clusters formed.

Fowlkes-Mallows Index (FMI) assesses clustering similarity based on pairs of samples correctly or incorrectly grouped together in both clusterings. FMI tends to yield higher values than ARI and NMI due to its focus on pairs of samples. However, it shares sensitivity to cluster count and dataset size with ARI.

In summary, each metric has strengths and limitations. ARI offers an intuitive scale but may produce high values for random clusterings. NMI provides normalized measures but may favor algorithms generating numerous clusters. FMI offers simplicity with a focus on sample pairs but shares sensitivity to cluster count and dataset size with ARI. Understanding these characteristics is crucial for selecting the most appropriate metric for evaluating clustering results.